In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tika


  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=60347db3f73ce5b3e0447012220fdf9ad4df830ffcd31e1ee6baacabea0a6303
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [ ]:
from tika import parser
import os
import json
import re
import pandas as pd

# **Extracting Text Data from CVs**

In [ ]:
# Function to preprocess a CV file
def preprocess_cv(cv_path):
    # Parse the CV file using Apache Tika
    parsed = parser.from_file(cv_path)

    # Extract text content and metadata
    text_content = parsed['content']
    metadata = parsed['metadata']

    # Extract relevant metadata fields (e.g., author, creation date, etc.)
    author = metadata.get('Author', 'Unknown')
    creation_date = metadata.get('Creation-Date', 'Unknown')

    # Create key-value pairs for the extracted metadata and text content
    key_value_pairs = {
        'Text Content': text_content
    }

    return key_value_pairs

# Directory containing the CV files in Google Drive
cv_directory = '/content/drive/MyDrive/Piramal Hackathon/dataset/CV test'

# Preprocess each CV file in the directory
cv_key_value_pairs = {}
for filename in os.listdir(cv_directory):
    if filename.endswith('.docx'):
        cv_path = os.path.join(cv_directory, filename)
        cv_key_value_pairs[filename] = preprocess_cv(cv_path)

# Save the key-value pairs to a JSON file
output_file = '/content/drive/MyDrive/Piramal Hackathon/cv_pairs.json'
with open(output_file, 'w') as f:
    json.dump(cv_key_value_pairs, f, indent=4)



2024-04-11 06:00:34,143 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2024-04-11 06:00:34,423 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2024-04-11 06:00:34,646 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


# **Cleaning of text from unwanted spaces,characters**

In [ ]:

def clean_text(text):
    # Remove newline characters and extra spaces, and return only words
    return ' '.join(text.split())

def clean_json(cv_key_value_pairs):
    cleaned_data = {}
    for filename, entry in cv_key_value_pairs.items():
        text = entry['Text Content']

        # Clean text content
        cleaned_text = re.sub(r'\s+', ' ', text.replace('\n', ' ')).strip()

        # Update entry with cleaned text content
        cleaned_entry = {
            'Text Content': cleaned_text
        }
        cleaned_data[filename] = cleaned_entry

    return cleaned_data

# Read the pairs from the JSON file
input_file = '/content/drive/MyDrive/Piramal Hackathon/cv_pairs.json'
with open(input_file, 'r') as f:
    cv_key_value_pairs = json.load(f)

# Clean the text content while keeping the format
cleaned_data = clean_json(cv_key_value_pairs)

# Save the cleaned data to a new JSON file
output_file = '/content/drive/MyDrive/Piramal Hackathon/cleaned_pairs1.json'
with open(output_file, 'w') as f:
    json.dump(cleaned_data, f, indent=4)

print("Cleaning complete. The cleaned data has been saved to", output_file)


Cleaning complete. The cleaned data has been saved to /content/drive/MyDrive/Piramal Hackathon/cleaned_pairs1.json


In [ ]:
import json
import re

def clean_text(text):
    # Remove newline characters, extra spaces, and return only words
    cleaned_text = ' '.join(text.split())
    # Remove Unicode characters
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)
    # Remove colons (':') and dashes ('-')
    cleaned_text = cleaned_text.replace(':', '').replace('-', '')
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

def clean_json(cv_key_value_pairs):
    cleaned_data = {}
    for filename, entry in cv_key_value_pairs.items():
        text = entry['Text Content']

        # Clean text content
        cleaned_text = clean_text(text)

        # Update entry with cleaned text content
        cleaned_entry = {
            'Text Content': cleaned_text
        }
        cleaned_data[filename] = cleaned_entry

    return cleaned_data

# Read the pairs from the JSON file
input_file = '/content/drive/MyDrive/Piramal Hackathon/cleaned_pairs1.json'
with open(input_file, 'r') as f:
    cv_key_value_pairs = json.load(f)

# Clean the text content while keeping the format
cleaned_data = clean_json(cv_key_value_pairs)

# Save the cleaned data to a new JSON file
output_file = '/content/drive/MyDrive/Piramal Hackathon/cleaned_pairs2.json'
with open(output_file, 'w') as f:
    json.dump(cleaned_data, f, indent=4)

print("Cleaning complete. The cleaned data has been saved to", output_file)


Cleaning complete. The cleaned data has been saved to /content/drive/MyDrive/Piramal Hackathon/cleaned_pairs2.json


# **Converting json to xlsx**

In [ ]:
# Read the JSON file into a DataFrame
input_file = '/content/drive/MyDrive/Piramal Hackathon/cleaned_pairs2.json'
df = pd.read_json(input_file)

# Define the output Excel file name
output_file = '/content/drive/MyDrive/Piramal Hackathon/employees.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(output_file, index=False)

print("Conversion complete. The Excel file has been saved to", output_file)

Conversion complete. The Excel file has been saved to /content/drive/MyDrive/Piramal Hackathon/employees.xlsx


# **Removing unwanted characters and spaces**

In [ ]:

# Load the Excel file
df = pd.read_excel('/content/drive/MyDrive/Piramal Hackathon/employees.xlsx')

# Remove text patterns like 'imageXX.png' or 'imageXXX.png'
df = df.replace(to_replace=r'image\d+\.png', value='', regex=True)

# Remove the occurrence of '}\n'
df = df.replace(to_replace=r'}\n', value='', regex=True)

# Remove any extra spaces
df = df.replace(to_replace=r'\s+', value=' ', regex=True)

# Save the cleaned data back to the Excel file
df.to_excel('/content/drive/MyDrive/Piramal Hackathon/cleaned_excel_file.xlsx',index=False)